<a href="https://colab.research.google.com/github/sleighton2022/datasci266-final-project/blob/main/Datasci_266_baseline_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datasci 266 final project - enhancement notebook - T5 Low Rank A

**Description:**

**Structure of the notebook:**

**Expected results:**





## Setup

In [131]:
!pip install -q sentencepiece
!pip install -q transformers
!pip install -q evaluate
!pip install -q datasets
!pip install -q pandas
!pip install -q rouge_score
!pip install -q nltk
!pip install bert_score
!pip install sentence-transformers
!pip install -q time
!pip install -q torchinfo

!pip install peft
!pip install accelerate

!pip install bitsandbytes==0.43.1



ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [132]:
#import evaluate
import torch
my_device = "cuda:0" if torch.cuda.is_available() else "cpu"
my_device

#from transformers import T5Tokenizer, TFT5ForConditionalGeneration
#from transformers import BertTokenizer, BertModel

import evaluate
import numpy as np
import time
from transformers import pipeline
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType

import bitsandbytes as bnb
from transformers import BitsAndBytesConfig


In [133]:
from datasets import load_dataset
xsum_dataset = load_dataset("xsum", trust_remote_code=True)

In [134]:
train_data = xsum_dataset['train']
train_data = train_data.shuffle(seed=42)
test_data = xsum_dataset['test']
test_data = test_data.shuffle(seed=42)
valid_data = xsum_dataset['validation']
valid_data = test_data.shuffle(seed=42)
print(len(train_data))
print(len(test_data))
print(train_data[0])


204045
11334
{'document': 'In Wales, councils are responsible for funding and overseeing schools.\nBut in England, Mr Osborne\'s plan will mean local authorities will cease to have a role in providing education.\nAcademies are directly funded by central government and head teachers have more freedom over admissions and to change the way the school works.\nIt is a significant development in the continued divergence of schools systems on either side of Offa\'s Dyke.\nAnd although the Welsh Government will get extra cash to match the money for English schools to extend the school day, it can spend it on any devolved policy area.\nMinisters have no plans to follow suit.\nAt the moment, governing bodies are responsible for setting school hours and they need ministerial permission to make significant changes.\nThere are already more than 2,000 secondary academies in England and its extension to all state schools is unlikely to shake the Welsh Government\'s attachment to what they call a "com

In [135]:
#subsample training and valid dataset
train_dataset = train_data.select(range(20000))#20000
valid_dataset = valid_data.select(range(500)) #500

#set test dataset for bart and refine bart models
test_dataset = test_data.select(range(200))

In [136]:
#global params
TOK_MAX_LEN = 1024
SUMMARY_MAX_LEN = 80
SUMMARY_MIN_LEN = 30


# T5 Model - LoRA


**set T5 model and inputs**

In [137]:
# shrink the 32 bit floating point number representations down to 4 bits
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)


In [138]:
!pip install bitsandbytes==0.43.1
model_checkpoint = 't5-base'
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint, quantization_config=nf4_config)
#model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)
#model = model.to(my_device)

embedding_layer = model.shared
max_tok_len = embedding_layer.weight.shape[1]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [154]:
# Tokenize the dataset
def tokenize_function(examples):
    inputs = [article for article in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_tok_len, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length= SUMMARY_MAX_LEN, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [155]:
tok_train = train_dataset.map(tokenize_function, batched=True)
tok_valid = valid_dataset.map(tokenize_function, batched=True)

**set LoRA parameters and config**

In [156]:
#Vary r (rank) in the experiments
r = 4
lora_alpha = r
lora_dropout = 0.1

In [157]:
#freeze all the model parameters
for param in model.parameters():
    param.requires_grad = False

    if param.ndim == 1:
        # cast the small parameters (e.g. layernorm) to fp32 for stability
        param.data = param.data.to(torch.float16)


In [158]:
#setup the lora config and call function to get the peft version of the model that adds the extra adapters and layers
peft_config = LoraConfig(
    inference_mode=False, task_type=TaskType.SEQ_2_SEQ_LM, r=r, lora_alpha=lora_alpha, lora_dropout=lora_dropout #
)

peft_config.use_cache = True #more efficient but takes more memory

In [159]:
model = get_peft_model(model, peft_config)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

import torch.nn as nn
class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

print('nb of trainable params, all params, :')
model.get_nb_trainable_parameters()

nb of trainable params, all params, :


(442368, 223345920)

**set up training arg and train**

In [160]:
from transformers import Trainer, TrainingArguments

In [161]:
import gc
gc.collect()

36927

In [ ]:
10# Set up the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs= 2,
    weight_decay=0.01,
    remove_unused_columns=False
)

# Define the Trainer
trainer = Trainer(
    model= model,
    args=training_args,
    train_dataset= tok_train,
    eval_dataset= tok_valid
)

start_time = time.time()
# train the model
trainer.train()
end_time = time.time()
train_time = end_time - start_time

metrics = trainer.evaluate()
print(metrics)

print(f'training time:  {round(train_time/60,2)} mins')


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.268400,1.051758


Epoch,Training Loss,Validation Loss
1,1.268400,1.051758


In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"total number of parameters in the model: {total_params}")

# count trainable parameters
trainable_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {trainable_parameters}")

**generate summary on test set**

In [164]:
PROMPT = 'summarize: '
t5_lora_summaries = []

start_time = time.time()
for article in test_dataset['document']:
    inputs = tokenizer(PROMPT + article, max_length= SUMMARY_MAX_LEN, truncation=True, return_tensors="pt")
    inputs = inputs.to(my_device)
    # Generate Summary
    summary_ids = model.generate(input_ids=inputs.input_ids,
                                         #max_new_tokens = max_tok_len,
                                         min_length= SUMMARY_MIN_LEN,
                                         max_length= SUMMARY_MAX_LEN,
                                         num_beams=4) #,
    candidate = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    t5_lora_summaries.append(candidate)
end_time = time.time()
inference_time = end_time - start_time

print(f'inference time:  {round(inference_time/60,2)} mins')

inference time:  10.2 mins


In [165]:
#check generated summary vs. reference
for i in range(len(test_dataset[:3])):

  print(test_dataset['summary'][i])
  print(t5_lora_summaries[i])
  print()

A woman who was seriously hurt in a fatal hen party motorway crash is now helping other major trauma victims rebuild their lives.
Sarah Johnson was one of 21 women heading to Liverpool when their minibus was hit by a lorry . her friend Bethany Jones, 18, was killed while Ms Johnson and several others were badly hurt .

A Tudor manor house has reopened following a £2.2m makeover.
1,400 tickets have sold out for the opening weekend of Bramall Hall in Stockport, Greater Manchester . stained glass windows and ceilings have been restored and the public can visit the dining room and butler's pantry for the first time .

Walt Disney World has unveiled a lighthouse memorial for a young boy who was killed by an alligator while on holiday at the Florida theme park.
Lane Thomas Graves, 2, was playing in the sand near the resort's Seven Seas Lagoon when he was dragged underwater .



**evaluation scores on test set**

In [166]:
#evaluation metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load('rouge')
bertscore = evaluate.load("bertscore")

from sentence_transformers import SentenceTransformer, util
sentence_transformer = SentenceTransformer("all-mpnet-base-v2")
def calculate_vector_similarity(reference_summaries, generated_summaries):
    """
    Calculates cosine similarity between reference and generated summary embeddings.
    """
    ref_embeddings = sentence_transformer.encode(reference_summaries, convert_to_tensor=True)
    gen_embeddings = sentence_transformer.encode(generated_summaries, convert_to_tensor=True)
    cosine_scores = util.cos_sim(ref_embeddings, gen_embeddings)

    avg_similarity = cosine_scores.diagonal().mean().item()  # Average cosine similarity
    return avg_similarity

In [167]:

references = test_dataset['summary']
candidates = t5_lora_summaries

rouge_outputs = rouge.compute(predictions=candidates, references=references)
print(rouge_outputs['rouge1'], rouge_outputs['rouge2'], rouge_outputs['rougeL'])
bleu_outputs = bleu.compute(predictions=candidates, references=references)
print(bleu_outputs['bleu'])
bert_outputs = bertscore.compute(predictions=candidates, references=references, lang="en")
print(np.mean(bert_outputs['f1']))
print(calculate_vector_similarity(references, candidates))
print()

0.17906435975983176 0.022355358432774204 0.13049581418017953
0.010714842431041356


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0.8539706203341484
0.4752300977706909



In [168]:
#check generated summary vs. reference
for i in range(len(test_dataset[:3])):

  print(test_dataset['summary'][i])
  print(t5_lora_summaries[i])
  print(len(test_dataset['summary'][i]))

A woman who was seriously hurt in a fatal hen party motorway crash is now helping other major trauma victims rebuild their lives.
Sarah Johnson was one of 21 women heading to Liverpool when their minibus was hit by a lorry . her friend Bethany Jones, 18, was killed while Ms Johnson and several others were badly hurt .
129
A Tudor manor house has reopened following a £2.2m makeover.
1,400 tickets have sold out for the opening weekend of Bramall Hall in Stockport, Greater Manchester . stained glass windows and ceilings have been restored and the public can visit the dining room and butler's pantry for the first time .
60
Walt Disney World has unveiled a lighthouse memorial for a young boy who was killed by an alligator while on holiday at the Florida theme park.
Lane Thomas Graves, 2, was playing in the sand near the resort's Seven Seas Lagoon when he was dragged underwater .
143
